# Delegation Unit Test
Performing verification of the delegation mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np

# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# print(module_path)
# sys.path.append(module_path)
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')


stakeDelegateds: 605 events
stakeDelegatedLockeds: 375 events
stakeDelegatedWithdrawns: 326 events
allocationCloseds: 47 events
allocationCollecteds: 16 events
stakeDepositeds: 25 events
rewardsAssigneds: 47 events
delegationParametersUpdateds: 6 events
allocationCreateds: 51 events
TOTAL NUMBER OF EVENTS: 1529
You should set SIMULATION_TIME_STEPS in config.py to a minimum of 1529 to capture all events.



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
# print(delegation_events)

getcontext().prec = 6
print(getcontext())
delegation_events_dict = delegation_events
print("EXPECTED TRUTH FROM DELEGATION EVENTS:")
for timestep, events in delegation_events_dict.items():
    for event in events:        
        print(f"{timestep}, {event['delegator']}, {event['shares']}")
print(getcontext())



Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
EXPECTED TRUTH FROM DELEGATION EVENTS:
3, 0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, 165851.1372
4, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.64465
7, 0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, 6965
8, 0x9327775e225a703125167eb423d246915bdc37e7, 9950
9, 0x32345932bc863f54ef87e2ece3d458f1789b1bfc, 1991.7586625
11, 0xb300749c08cab1c9ce67f66a18ea5926360ba36a, 503.90508365
12, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.075
15, 0xb7797cb7b4a031c4c1e04ea14824fa5d8d30beed, 39800
16, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.5676239939
17, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.714085664846265
20, 0x1beda87a78e838687668ce2c564f1746dce7396e, 765384.845
22, 0x7eb692f22af33d637aab31f9277653235682ebe6, 4002.8933795146576425
23, 0xb4ecbb9d116a2b38cee4ede75b0a96746a175efb, 9950
24, 0x4e43a0cd79a9324645167e70aa6cdb

In [4]:
timestep = 1
# df.iloc[timestep-1].delegators['0xd776a7306ee6a060cebb46b46d305e88fd39ba84'].shares
# df.iloc[timestep*6+1].delegators

# timestep += 1
# df.iloc[3].delegators['0xd776a7306ee6a060cebb46b46d305e88fd39ba84']
print("MODELED RESULTS")
print(getcontext())
delegation_event_shares = {}
for timestep, events in delegation_events_dict.items():
    for event in events:
        # curTimestepShares = df.iloc[timestep-1].delegators[event['delegator']].shares
        curTimestepShares = df.iloc[timestep-1].indexers[event['indexer']].delegators[event['delegator']].shares
        try:
            # lastTimestepShares = df.iloc[timestep-2].delegators[event['delegator']].shares
            lastTimestepShares = df.iloc[timestep-2].indexers[event['indexer']].delegators[event['delegator']].shares
        except:
            lastTimestepShares = 0
        deltaShares = curTimestepShares - lastTimestepShares
        delegation_event_shares[timestep] = deltaShares
        print(f"{timestep}, {event['delegator']}, {delegation_event_shares[timestep]}")



MODELED RESULTS
Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
3, 0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, 165851.1372
4, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.6446500000000000000001
7, 0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, 6965.000000000000000000000003
8, 0x9327775e225a703125167eb423d246915bdc37e7, 9950.000000000000000000000003
9, 0x32345932bc863f54ef87e2ece3d458f1789b1bfc, 1991.758662500000000000000000
11, 0xb300749c08cab1c9ce67f66a18ea5926360ba36a, 503.9050836500000000000000003
12, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.07500000000000000000000
15, 0xb7797cb7b4a031c4c1e04ea14824fa5d8d30beed, 39800.00000000000000000000001
16, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.56762399390000000000000
17, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.71408566484626500000000
20, 0x1beda87a78e838687668ce2c564f1746dce7396e, 765384.8450000000000000000

In [5]:
print("UNITTEST RESULTS")
print(getcontext())
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, events in delegation_events_dict.items():
    for event in events:
        # curTimestepShares = df.iloc[timestep-1].delegators[event['delegator']].shares
        curTimestepShares = df.iloc[timestep-1].indexers[event['indexer']].delegators[event['delegator']].shares
        try:
            # lastTimestepShares = df.iloc[timestep-2].delegators[event['delegator']].shares
            lastTimestepShares = df.iloc[timestep-2].indexers[event['indexer']].delegators[event['delegator']].shares
        except:
            # this is a new delegator.
            lastTimestepShares = 0
        deltaShares = curTimestepShares - lastTimestepShares

        print(f"Event: Timestep={timestep}, Delegator ID={event['delegator']}, Shares Granted={event['shares']}")
        print(f"Model: Timestep={timestep}, Delegator ID={event['delegator']}, Shares Granted={deltaShares}")
        
        try:
            ratio = abs(event['shares'] / deltaShares)
        except:
            ratio = np.inf
        exact = ratio == 1.0
        close = 0.99 <= ratio <= 1.01
        reallyClose = 0.99999 <= ratio <= 1.00001

        if exact:            
            cntExact += 1
        elif reallyClose:
            cntReallyClose += 1
        elif close:
            cntClose += 1
        else:
            cntWrong += 1
        cnt += 1
        print("Shares Equal?", exact)
        print("Shares Within 0.001%?", reallyClose)
        print("Shares Within 1%?", close)
        print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
        print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   


UNITTEST RESULTS
Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
Event: Timestep=3, Delegator ID=0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, Shares Granted=165851.1372
Model: Timestep=3, Delegator ID=0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, Shares Granted=165851.1372
Shares Equal? True
Shares Within 0.001%? True
Shares Within 1%? True
Exact: 1, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 1

Event: Timestep=4, Delegator ID=0xd776a7306ee6a060cebb46b46d305e88fd39ba84, Shares Granted=251819.64465
Model: Timestep=4, Delegator ID=0xd776a7306ee6a060cebb46b46d305e88fd39ba84, Shares Granted=251819.6446500000000000000001
Shares Equal? False
Shares Within 0.001%? True
Shares Within 1%? True
Exact: 1, ReallyClose: 1, Close: 0, Wrong: 0, Total Number: 2

Event: Timestep=7, Delegator ID=0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, Shares Granted=6965
Model: Timestep=7, Delegator ID=0x53c0

# Conclusion
The number of shares granted by delegation in the first two events match the output of the cadCAD model exactly and then the next 3 are extremely close, diverging gradually.